In [3]:
import os 
import numpy as np 
import pandas as pd 
import sagemaker 

  

sagemaker_session = sagemaker.Session() 

bucket = "sagemaker-studio-772149141904-nrs4q1pu91" 
prefix = "sagemaker/pytorch-exoplanets" 

role = sagemaker.get_execution_role() 

x_inputs_train = sagemaker_session.upload_data("./data/data_train_x_scaled_filt.txt", bucket=bucket, key_prefix=prefix) 
y_inputs_train = sagemaker_session.upload_data("./data/data_train_y.csv", bucket=bucket, key_prefix=prefix) 


x_inputs_test = sagemaker_session.upload_data("./data/data_test_x_scaled_filt.txt", bucket=bucket, key_prefix=prefix) 
y_inputs_test = sagemaker_session.upload_data("./data/data_test_y.csv", bucket=bucket, key_prefix=prefix) 

In [4]:
x_inputs_train

's3://sagemaker-studio-772149141904-nrs4q1pu91/sagemaker/pytorch-exoplanets/data_train_x_scaled_filt.txt'

In [5]:
import boto3

output_path = f"s3://{bucket}/{prefix}" 

In [7]:
!aws s3 ls --recursive {output_path+"/"}

2022-03-17 15:07:45   93461749 sagemaker/pytorch-exoplanets/data_test_x_scaled_filt.txt
2022-03-17 15:07:46       1146 sagemaker/pytorch-exoplanets/data_test_y.csv
2022-03-17 15:07:38  833955941 sagemaker/pytorch-exoplanets/data_train_x_scaled_filt.txt
2022-03-17 15:07:45      10180 sagemaker/pytorch-exoplanets/data_train_y.csv
2022-03-15 20:25:01    1596994 sagemaker/pytorch-exoplanets/pytorch-training-2022-03-15-20-20-00-797/profiler-output/system/incremental/2022031520/1647375780.algo-1.json
2022-03-15 20:25:05    1639595 sagemaker/pytorch-exoplanets/pytorch-training-2022-03-15-20-20-00-797/profiler-output/system/incremental/2022031520/1647375840.algo-1.json
2022-03-15 20:25:05     123040 sagemaker/pytorch-exoplanets/pytorch-training-2022-03-15-20-20-00-797/profiler-output/system/incremental/2022031520/1647375900.algo-1.json
2022-03-15 20:25:28     340486 sagemaker/pytorch-exoplanets/pytorch-training-2022-03-15-20-20-00-797/rule-output/ProfilerReport-1647375600/profiler-output/profi

In [9]:
from sagemaker.pytorch import PyTorch 

estimator = PyTorch( 
    entry_point="train.py", 
    source_dir="code", 
    role=role, 
    framework_version="1.6", 
    py_version="py3", 
    instance_count=1,   
    instance_type="ml.g4dn.12xlarge", 
    output_path=output_path, 
    hyperparameters={ 
        "epochs": 10, 
        "lr" : 1e-5,  
        "batch-size" : 24,
        "test-batch-size" : 16,
        "log-interval": 50,
        "num-gpus": 1,
        "model-name": "torch_cnn"
    } 

) 

estimator.fit({"training": 's3://sagemaker-studio-772149141904-nrs4q1pu91/sagemaker/pytorch-exoplanets/data_', 
               "testing": 's3://sagemaker-studio-772149141904-nrs4q1pu91/sagemaker/pytorch-exoplanets/test_'}) 

2022-03-17 15:47:34 Starting - Starting the training job...
2022-03-17 15:47:57 Starting - Launching requested ML instancesProfilerReport-1647532054: InProgress
...............
2022-03-17 15:50:28 Starting - Preparing the instances for training......
2022-03-17 15:51:26 Downloading - Downloading input data...
2022-03-17 15:51:58 Training - Downloading the training image........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-03-17 15:53:17,805 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-03-17 15:53:17,846 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-03-17 15:53:20,865 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-03-17 15:53:21,255 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "te

In [59]:
from sagemaker.pytorch.model import PyTorchModel 

model_data = estimator.model_data 


from sagemaker.pytorch.model import PyTorchModel 

model_data = estimator.model_data 

pytorch_model = PyTorchModel(model_data=model_data, 
                             role=role, 
                             framework_version="1.6", 
                             source_dir="code", 
                             py_version="py3", 
                             entry_point="inference.py") 

  
predictor = pytorch_model.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge") 


--------!

In [60]:
predictor.serializer = sagemaker.serializers.JSONSerializer() 
predictor.deserializer = sagemaker.deserializers.JSONDeserializer() 

In [20]:
import json
arr = np.random.randn(3197) * 30000
arr = arr.tolist()
json.dumps(arr)

'[-4912.3557952646315, 21643.568768950925, -41147.15831527094, -51314.187717676126, 39154.82071770666, 63589.3419619149, -29168.80144178919, 58472.13214402983, 7559.150992732426, -40607.59841376488, 26329.163662467232, 10922.38998607109, 10239.913545664853, 11034.625160587011, -583.7521791595269, 20900.74452585729, -25736.355436194364, -21188.25496159106, 8371.290985965055, 33265.758320879206, -20623.176166525536, 21812.31126436585, 32753.40238912537, 30087.675660727862, 3325.75259104134, -18687.419731983162, 17265.769144575166, -59595.599305810356, -2217.5410866488864, -51448.064926445186, -4586.641714643178, -8229.488264282174, -29841.193483069837, -9936.543785998092, 28138.00934998991, 27476.26543829074, -5819.6195701770575, 36857.99049246157, 26346.240158687942, 18288.06008814884, -13679.298606693217, 13287.459460955633, -33407.462565232156, -30238.745518337622, -11967.46493378572, -51309.69348636862, 6737.15568205184, 19420.903805445883, 25575.741759559394, -63707.071076244916, -6

In [55]:
import pickle
from scipy.ndimage import uniform_filter1d

def prep_input(data):
    scaler = pickle.load(open('./data/scaler.pkl', 'rb'))
    
    if isinstance(data, list):
        data = np.array(data)
    
    data = data[np.newaxis, :] 
    data = scaler.transform(data)
    
    # pad shorter arrays
    INPUT_LEN = 3197
    if len(data) < INPUT_LEN:
        padded = np.zeros(INPUT_LEN)
        for i, p in enumerate(data):
            padded[i] = p[i]
    if len(data) > INPUT_LEN:
        padded = data[:-INPUT_LEN]
    else:
        data = padded
    
    padded = padded[np.newaxis, :] 
    input_data = np.stack([padded, uniform_filter1d(padded, axis=1, size=200)], axis=2)
    shape = input_data.shape
    input_data = input_data.reshape(shape[0], shape[2], shape[1])
    
    return input_data

In [56]:
input_data = prep_input(arr)

In [57]:
input_data.shape

(1, 2, 3197)

In [58]:
input_data= input_data.tolist()

In [62]:
json.dumps(input_data)

'[[[-0.23515283632040207, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.0023515283632040205, 0.0, -0.00235152836320

In [63]:
predictor.predict(json.dumps(input_data))

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from primary with message "Your invocation timed out while waiting for a response from container primary. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-west-2.console.aws.amazon.com/cloudwatch/home?region=us-west-2#logEventViewer:group=/aws/sagemaker/Endpoints/pytorch-inference-2022-03-17-17-21-27-897 in account 772149141904 for more information.

In [64]:
predictor.delete_endpoint()